In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence

from keras.layers import LSTM, Conv1D, MaxPooling1D
from keras.layers import Dense, Dropout, Activation
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

Using TensorFlow backend.


In [2]:
max_features = 10000
maxlen = 500
batch_size = 32

In [3]:
(x_train, y_train), (x_label, y_label) = imdb.load_data(num_words = max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_label = sequence.pad_sequences(x_label, maxlen=maxlen)

Sequential모델로 (10000, 50)의 Embedding을 추가해준다. Embedding이 레이어 중간에 있으면
앞의 레이어에서 출력되는 크기를 알기때문에 input_length를 반드시 추가해야할 필요는 없지만, 이 모델에서는 Embedding이 
첫 레이어에 위치하므로 input_length를 적어준다.

분류하는 모델이기에 Dense 1과 함께 활성화함수 sigmoid를 사용한다.

In [4]:
model = Sequential()
model.add(Embedding(max_features, 50, input_length=maxlen))
# 50dimension
model.add(Dropout(0.2)) 
model.add(Conv1D(50, 5,
                padding='valid',
                activation='relu',
                strides=1))
model.add(MaxPooling1D(pool_size=3))
model.add(LSTM(32))

model.add(Dense(1))
model.add(Activation('sigmoid'))

W0803 13:40:58.108543 139946534573888 deprecation_wrapper.py:119] From /home/aaronroh/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0803 13:40:58.121585 139946534573888 deprecation_wrapper.py:119] From /home/aaronroh/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0803 13:40:58.124550 139946534573888 deprecation_wrapper.py:119] From /home/aaronroh/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0803 13:40:58.135916 139946534573888 deprecation_wrapper.py:119] From /home/aaronroh/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.p

In [5]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])

W0803 13:40:58.383102 139946534573888 deprecation_wrapper.py:119] From /home/aaronroh/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0803 13:40:58.404922 139946534573888 deprecation_wrapper.py:119] From /home/aaronroh/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0803 13:40:58.410186 139946534573888 deprecation.py:323] From /home/aaronroh/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


x_train, y_train으로 훈련을 하며, 검증을 위한 데이터로 x_label, y_label을 사용하였다.

In [6]:
history = model.fit(x_train, y_train,
                   epochs=25,
                   batch_size = 128,
                validation_data=(x_label, y_label)
                   )

Train on 25000 samples, validate on 25000 samples
Epoch 1/25
25000/25000 [==============================] - 42s 2ms/step - loss: 0.4475 - acc: 0.7820 - val_loss: 0.3507 - val_acc: 0.8503
Epoch 2/25
25000/25000 [==============================] - 44s 2ms/step - loss: 0.2586 - acc: 0.8992 - val_loss: 0.2921 - val_acc: 0.8805
Epoch 3/25
25000/25000 [==============================] - 43s 2ms/step - loss: 0.2101 - acc: 0.9207 - val_loss: 0.2855 - val_acc: 0.8813
Epoch 4/25
25000/25000 [==============================] - 45s 2ms/step - loss: 0.1749 - acc: 0.9370 - val_loss: 0.3040 - val_acc: 0.8752
Epoch 5/25
25000/25000 [==============================] - 40s 2ms/step - loss: 0.1522 - acc: 0.9449 - val_loss: 0.3284 - val_acc: 0.8763
Epoch 6/25
25000/25000 [==============================] - 39s 2ms/step - loss: 0.1307 - acc: 0.9538 - val_loss: 0.4776 - val_acc: 0.8374
Epoch 7/25
25000/25000 [==============================] - 39s 2ms/step - loss: 0.1153 - acc: 0.9603 - val_loss: 0.3671 - val_acc

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 50)           500000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 50)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 496, 50)           12550     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 165, 50)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                10624     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total para

In [8]:
results = model.evaluate(x_label, y_label)

25000/25000 [==============================] - 14s 569us/step


In [9]:
results

[0.8606499513095617, 0.84608]

0.84608의 정확도가 나왔다.